In [ ]:
import pandas as pd

gene_name = "BTNL3"

import pysam

# Open the BCF file
vcf_file = pysam.VariantFile("/home/tl688/pitl688/enformer_fine_tuning/data/VCF/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze_SNPsOnly.bcf.gz", "rb") 

# Iterate over the variants
variant_list = []
for variant in vcf_file:
    variant_list.append(variant.id)
    break
# Close the file
vcf_file.close()

data = sorted(variant.samples.keys())

count = 0
for i in data:
    if i == 'GTEX-1LG7Y':
        print(count)
    count +=1

gene_exp = pd.read_csv("/home/tl688/pitl688/enformer_fine_tuning/data/gtex_eqtl_expression_matrix/Whole_Blood.v8.normalized_expression_genesymbol.csv", sep='\t')

df_gene_test = gene_exp[gene_exp['gene_symbol'].isin([gene_name])]
idx_gene = df_gene_test.index

idx_gene

sample_name = list(variant.samples.keys())

# sample_name

import sklearn.linear_model

gene_exp = pd.read_csv("/home/tl688/pitl688/enformer_fine_tuning/data/gtex_eqtl_expression_matrix/Whole_Blood.v8.normalized_expression_genesymbol.csv", sep='\t')

import torch

data_sample = torch.load(f"/gpfs/radev/project/ying_rex/tl688/seq2cells_code/tests/{gene_name}_ukbiobert_emb_concat_49000_mean.pkl")

data_sample = data_sample.numpy()

In [ ]:
# df_waittrain = pd.DataFrame(data_sample)

data_sample_new = (data_sample[:,0:1536//2] + data_sample[:,1536//2:]) / 2
df_waittrain = pd.DataFrame(data_sample_new)

inter_sample = sorted(set(list(gene_exp.columns[4:])).intersection(set(sample_name)))

df_train_update = df_waittrain
df_train_update.index = inter_sample

df_train_update.index

gene_exp_train = gene_exp[inter_sample]

gene_exp_train

import scanpy as sc

adata = sc.AnnData(df_waittrain.values)

adata.obs_names = list(gene_exp_train.columns)

gene_exp_train.loc[idx_gene].values.shape

adata.obs['expression'] = gene_exp_train.loc[idx_gene].values[0]

In [ ]:
adata

In [ ]:

sc.tl.pca(adata)
sc.pp.neighbors(adata)

sc.tl.umap(adata)

sc.pl.umap(adata, color='expression')